In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from keras.models import load_model
import os
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.applications.inception_v3 import InceptionV3
import os
from keras.layers import Flatten, Dense, AveragePooling2D
from keras.models import Model
from keras.optimizers import RMSprop, SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator


from keras.layers import merge
from keras.layers.core import Lambda
from keras.models import Model
import tensorflow as tf

import make_parallel

Using TensorFlow backend.


In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

img_width = 299
img_height = 299
batch_size = 32
nbr_test_samples = 1000
nbr_augmentation = 5

FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

root_path = '/home/ubuntu/jupyter/Fish/Final_Project/'

weights_path = os.path.join(root_path, 'inception_weights.h5')

test_data_dir = os.path.join(root_path, 'test_stg1/')

# test data generator for prediction
test_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)

In [5]:
learning_rate = 0.0001
img_width = 299
img_height = 299
nbr_train_samples = 3016
nbr_validation_samples = 756
nbr_epochs = 25
batch_size = 64

train_data_dir = '/home/ubuntu/jupyter/Fish/Final_Project/train2/train_split'
val_data_dir = '/home/ubuntu/jupyter/Fish/Final_Project/train2/val_split'

FishNames = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

print('Loading InceptionV3 Weights ...')
InceptionV3_notop = InceptionV3(include_top=False, weights='imagenet', input_tensor=None, input_shape = (3, 299,299))
# Note that the preprocessing of InceptionV3 is:
# (x / 255 - 0.5) x 2

print('Adding Average Pooling Layer and Softmax Output Layer ...')
output = InceptionV3_notop.get_layer(index = -1).output  # Shape: (8, 8, 2048)
output = AveragePooling2D((8, 8), strides=(8, 8), name='avg_pool')(output)
output = Flatten(name='flatten')(output)
output = Dense(8, activation='softmax', name='predictions')(output)

InceptionV3_model = Model(InceptionV3_notop.input, output)
#InceptionV3_model.summary()

InceptionV3_model = make_parallel(InceptionV3_model,4)
optimizer = SGD(lr = learning_rate, momentum = 0.9, decay = 0.0, nesterov = True)
InceptionV3_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
InceptionV3_model.load_weights("/home/ubuntu/jupyter/Fish/Final_Project/inception_weights.h5")

Loading InceptionV3 Weights ...


/usr/local/lib/python2.7/site-packages/keras/applications/inception_v3.py:291: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image dimension ordering convention (`image_dim_ordering="th"`). For best performance, set `image_dim_ordering="tf"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


Adding Average Pooling Layer and Softmax Output Layer ...


In [6]:
for idx in range(10):
    print('{}th augmentation for testing ...'.format(idx))
    random_seed = np.random.random_integers(0, 100000)

    test_generator = test_datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=batch_size,
            shuffle = False, # Important !!!
            seed = random_seed,
            classes = None,
            class_mode = None)

    test_image_list = test_generator.filenames
    #print('image_list: {}'.format(test_image_list[:10]))
    print('Begin to predict for testing data ...')
    if idx == 0:
        predictions = InceptionV3_model.predict_generator(test_generator, nbr_test_samples)
    else:
        predictions += InceptionV3_model.predict_generator(test_generator, nbr_test_samples)

predictions /= nbr_augmentation

print('Begin to write submission file ..')
f_submit = open(os.path.join(root_path, 'submit.csv'), 'w')
f_submit.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
for i, image_name in enumerate(test_image_list):
    pred = ['%.6f' % p for p in predictions[i, :]]
    if i % 100 == 0:
        print('{} / {}'.format(i, nbr_test_samples))
    f_submit.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))

f_submit.close()

print('Submission file successfully generated!')

0th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: This function is deprecated. Please call randint(0, 100000 + 1) instead
  app.launch_new_instance()


1th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
2th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
3th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
4th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
5th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
6th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
7th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
8th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for testing data ...
9th augmentation for testing ...
Found 1000 images belonging to 1 classes.
Begin to predict for 